### Import Useful Libraries

In [17]:
import pandas as pd
import numpy as np
import time
import ast
from sklearn.metrics import hamming_loss, f1_score ,classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import KFold
from google.colab import drive

start_time = time.time()
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Rerieve Dataset

In [18]:
max = 20
path = '/content/drive/MyDrive/Satori Assignment/data/preprocessed_data_'+ str(max) +'.csv'
df = pd.read_csv(path)
df['Target'] = df['Target'].apply(ast.literal_eval)
df['Tag'] = df['Tag'].apply(ast.literal_eval)

print(df.shape)
display(df.head())

(33374, 5)


,Text,Tag,Num_Tags,Num_Words,Target
0,mysql error php mysql query mysql error php my...,"[mysql, php]",2,138,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]"
1,setup site-wide variables php setup site-wide ...,[php],1,51,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
2,select select select way select data one multi...,[mysql],1,31,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
3,comes comes comes searched little gotten parti...,[c],1,46,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,code ask yes/no question javascript code ask y...,[javascript],1,26,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"


> We load the preprocessed_data_K CSV file, which stores data after preprocessing with the Top K Tag Combinations.

In [19]:
unique_tags = set([tag for sublist in df['Tag'] for tag in sublist])

print(f"Number of Unique Tags: {len(unique_tags)}")
print(unique_tags)

Number of Unique Tags: 16
{'angularjs', 'html', 'r', 'javascript', 'python', 'c#', 'java', 'android', 'c', 'php', 'mysql', 'sql', 'jquery', 'git', 'css', 'c++'}


### Split data

#### Convert Multilabel to Single-Label (for Split Purposes)

In [20]:
def calculate_tag_combinations(temp_df):
    """
    Create a dataframe 'tag_combination', which stores the unique combination of Tags and the number of appearance
    """

    if temp_df.empty:
        print('Error: Provided DataFrame is empty!')
        return

    temp_df['Tag'] = temp_df['Tag'].apply(sorted)
    temp_df['Tag_Tuple'] = temp_df['Tag'].apply(tuple)
    tag_combinations = temp_df.groupby('Tag_Tuple').size().reset_index(name='Count')
    temp_df.drop(columns=['Tag_Tuple'], inplace=True)
    tag_combinations = tag_combinations.sort_values(by='Count', ascending=False)

    sum_count = tag_combinations['Count'].sum()
    all_tags = [tag for tag_tuple in tag_combinations['Tag_Tuple'] for tag in tag_tuple]
    num_unique_tags = len(set(all_tags))

    print(f"Number of Questions of Tag Combinations: {sum_count}")
    print(f"Number of unique tags of Tag Combinations: {num_unique_tags}")
    print(f"Minimum Frequency of Tag Combinations: {tag_combinations['Count'].iloc[-1]}\n")
    display(tag_combinations)

    return tag_combinations

original_tag_combinations = calculate_tag_combinations(df)

Number of Questions of Tag Combinations: 33374
Number of unique tags of Tag Combinations: 16
Minimum Frequency of Tag Combinations: 823



,Tag_Tuple,Count
0,"(android,)",3630
12,"(javascript, jquery)",2700
10,"(java,)",2519
11,"(javascript,)",2501
17,"(php,)",2190
18,"(python,)",2109
4,"(c#,)",1944
13,"(jquery,)",1898
6,"(css, html)",1879
5,"(c++,)",1608


> Display in a proper way the distribution of Tag Combinations. This will be helpful so as to reassure in a given dataframe data are well distributed and randomly picken. This is a way to convert multilabel problem to single class problem and manually split the data on Tag Combinations and not on each single Tag.

In [21]:
def split_data(tag_combinations, df, create_val=True, train_ratio=0.8, val_ratio=0.1):
    # Shuffle the dataframe
    df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

    val_ratio = val_ratio if create_val else 0

    # Dicts to store counts for each combination
    train_counts = {combination: 0 for combination in tag_combinations['Tag_Tuple']}
    val_counts = {combination: 0 for combination in tag_combinations['Tag_Tuple']}
    test_counts = {combination: 0 for combination in tag_combinations['Tag_Tuple']}

    # Appropriate ratio of tag combinations for each set
    for index, row in tag_combinations.iterrows():
        combination = row['Tag_Tuple']
        count = row['Count']

        train_counts[combination] += int(count*train_ratio)
        val_counts[combination] += int(count*val_ratio)
        test_counts[combination] += count - int(count*train_ratio) - int(count*val_ratio)

    # Store data to list for each set
    train_data, val_data, test_data = [], [], []

    # Iterate over rows in df and assign to train, val, or test sets
    for index, row in df_shuffled.iterrows():
        combination = row['Tag']
        if train_counts[tuple(combination)] > 0:
            train_data.append(row)
            train_counts[tuple(combination)] -= 1
        elif test_counts[tuple(combination)] > 0:
            test_data.append(row)
            test_counts[tuple(combination)] -= 1
        elif val_counts[tuple(combination)] > 0:
            val_data.append(row)
            val_counts[tuple(combination)] -= 1

    assert len(train_data) + len(val_data) + len(test_data) == len(df), "Total number of examples in train, val, and test sets does not match the total number of examples in the original dataframe"
    return pd.DataFrame(train_data), pd.DataFrame(val_data), pd.DataFrame(test_data)

train_df, val_df, test_df = split_data(original_tag_combinations, df, create_val=False)

> Now with this generic function we utilise a logic with dicts in order to track the number of times of each Tag Combination and finally retrieve with a modifiable ratio train, test (and it is desired validation) sets.

#### Train, Val and Test Dataframes

In [22]:
train_df

,Text,Tag,Num_Tags,Num_Words,Target
0,echo php html line echo php html line echo php...,[php],1,37,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
1,aligning list links right aligning list links ...,"[css, html]",2,150,"[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
2,sort unordered list inside anchor tag sort uno...,[jquery],1,139,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]"
3,iso c99 intypes.h printf print hex value uint6...,[c],1,70,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,php unique array php unique array php unique a...,[php],1,77,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
...,...,...,...,...,...
27287,trouble input objects trouble input objects tr...,"[html, javascript, jquery]",3,678,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0]"
27370,chk span tag contain text chk span tag contain...,"[html, javascript, jquery]",3,130,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0]"
27393,remove image tags inside anchor tags remove im...,"[html, javascript, jquery]",3,201,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0]"
27396,click element traversing across containers cli...,"[html, javascript, jquery]",3,188,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0]"


In [23]:
test_df

,Text,Tag,Num_Tags,Num_Words,Target
26154,saving restoring state android saving restorin...,"[android, java]",2,127,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
26163,callback android callback android callback and...,"[android, java]",2,33,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
26176,selecting next sibling .parent selecting next ...,"[javascript, jquery]",2,116,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]"
26183,caching jquery variables caching jquery variab...,"[javascript, jquery]",2,41,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]"
26185,would want use jquery would want use jquery wo...,"[javascript, jquery]",2,106,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]"
...,...,...,...,...,...
33369,stop duplication data left join stop duplicati...,"[mysql, sql]",2,131,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1]"
33370,python redefining function within function pyt...,[python],1,102,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
33371,make defensive copies getters inside immutable...,[java],1,51,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
33372,polymorph collection c++ polymorph collection ...,[c++],1,71,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


#### Reassure the well distributed split

In [24]:
train_tag_combinations = calculate_tag_combinations(train_df)

Number of Questions of Tag Combinations: 26692
Number of unique tags of Tag Combinations: 16
Minimum Frequency of Tag Combinations: 658



,Tag_Tuple,Count
0,"(android,)",2904
12,"(javascript, jquery)",2160
10,"(java,)",2015
11,"(javascript,)",2000
17,"(php,)",1752
18,"(python,)",1687
4,"(c#,)",1555
13,"(jquery,)",1518
6,"(css, html)",1503
5,"(c++,)",1286


In [25]:
test_tag_combinations = calculate_tag_combinations(test_df)

Number of Questions of Tag Combinations: 6682
Number of unique tags of Tag Combinations: 16
Minimum Frequency of Tag Combinations: 165



,Tag_Tuple,Count
0,"(android,)",726
12,"(javascript, jquery)",540
10,"(java,)",504
11,"(javascript,)",501
17,"(php,)",438
18,"(python,)",422
4,"(c#,)",389
13,"(jquery,)",380
6,"(css, html)",376
5,"(c++,)",322


> We are pretty sure right now that each combination is well distributed and thus this will lead to better model's performance. Balanced datasets enable models with way more probabilities to capture patterns.

### TFidf Feature Extraction

In [26]:
tfidf_vectorizer = TfidfVectorizer(max_features=20000, ngram_range=(1,3))

X_train = tfidf_vectorizer.fit_transform(train_df['Text'])
Y_train = np.array(train_df['Target'].tolist())
print('X_train shape:', X_train.shape)
print('Y_train shape:', Y_train.shape)

X_test = tfidf_vectorizer.transform(test_df['Text'])
Y_test = np.array(test_df['Target'].tolist())
print('X_test shape:', X_test.shape)
print('Y_test shape:', Y_test.shape)

X_train shape: (26692, 20000)
Y_train shape: (26692, 16)
X_test shape: (6682, 20000)
Y_test shape: (6682, 16)


So we apply TFIDF with max_features=20000 and considering unigrams,bi-grams and tri-grams, cause there are many important combinations like 'unix environment' or 'android application'.

### Sklearn Models

> In order to experiment with different Sklearn models, create a generic function to monitor each algorithm's performance.

In [27]:
def train_and_evaluate_model(model, model_name):
    start_training = time.time()
    # Train and Predict
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)

    # Evaluation metrics
    hamming_loss_val = round(hamming_loss(Y_test, Y_pred), 2)
    micro_f1 = round(f1_score(Y_test, Y_pred, average='micro'), 2)
    macro_f1 = round(f1_score(Y_test, Y_pred, average='macro'), 2)

    print("-" * 90)
    print(f"> {model_name}")
    print(f"Hamming Loss: {hamming_loss_val}\tMicro-F1: {micro_f1}\t\tMacro-F1: {macro_f1}\t\tTime: {int((time.time()-start_training))} secs")

    return Y_pred

> **[Hamming Loss](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.hamming_loss.html)**: Hamming loss measures the fraction of labels that are incorrectly predicted. It treats each label prediction independently and penalizes each misclassification equally.

> **[Micro-F1 Score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)**: This metric computes the F1 score for each label individually and then averages them using the micro-average strategy. It is suitable for imbalanced datasets where some labels may have significantly fewer instances than others. Micro-F1 gives equal weight to each instance and each label. Micro-F1 is suitable when you want to evaluate the overall performance of the model without considering label imbalances.

> **[Macro-F1 Score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)**: Macro-F1 computes the F1 score for each label individually and then averages them using the macro-average strategy. Each label contributes equally to the average, regardless of the number of instances. Macro-F1 is useful when you want to evaluate the performance of the model on each label separately and give equal importance to each label.

> Using these three metrics together provides a **holistic** view of the model's performance in multi-label classification problem, considering different aspects such as overall accuracy, label-wise accuracy, and the balance between precision and recall.






> The Hamming loss is the fraction of labels that are incorrectly predicted. Ranges from [0,1] and it's logic is the smaller the better. With the 2 following simple and concrete examples it would be easier to understand.

----

    example_pred = [0, 0, 0, 0, 1]
    example_true = [0, 0, 0, 1, 0]
    hamming_loss(example_true, example_pred)
    
> In the above example, Hamming Loss = 0.4.

----

    example_pred = [0, 0, 0, 0, 1]
    example_true = [0, 0, 0, 1, 1]
    hamming_loss(example_true, example_pred)
    
> In the above example, Hamming Loss = 0.2.

----

    

In [28]:
init = time.time()

# Logistic Regression
reg_model = OneVsRestClassifier(LogisticRegression(max_iter=1000))
train_and_evaluate_model(reg_model, 'Logistic Regression')

# Stohastic Gradient Descent
sgd_model = OneVsRestClassifier(SGDClassifier())
train_and_evaluate_model(sgd_model, 'SGD')

# Linear SVC
svc_model = OneVsRestClassifier(LinearSVC())
Y_pred = train_and_evaluate_model(svc_model, 'Linear SVC')

# Multinomial Naive Bayes
nb_model = OneVsRestClassifier(MultinomialNB(alpha=1.0))
train_and_evaluate_model(nb_model, 'Multinomial Naive Bayes')

# Random Forest
rf_model = RandomForestClassifier(n_estimators=100)
train_and_evaluate_model(rf_model, 'Random Forest')

print("-" * 90)
print("\nCell Runtime:", int((time.time()-init)/60), "minutes")

------------------------------------------------------------------------------------------
> Logistic Regression
Hamming Loss: 0.03	Micro-F1: 0.79		Macro-F1: 0.74		Time: 27 secs
------------------------------------------------------------------------------------------
> SGD
Hamming Loss: 0.03	Micro-F1: 0.81		Macro-F1: 0.75		Time: 3 secs
------------------------------------------------------------------------------------------
> Linear SVC
Hamming Loss: 0.03	Micro-F1: 0.83		Macro-F1: 0.81		Time: 4 secs
------------------------------------------------------------------------------------------
> Multinomial Naive Bayes
Hamming Loss: 0.05	Micro-F1: 0.7		Macro-F1: 0.6		Time: 0 secs
------------------------------------------------------------------------------------------
> Random Forest
Hamming Loss: 0.03	Micro-F1: 0.8		Macro-F1: 0.73		Time: 128 secs
------------------------------------------------------------------------------------------

Cell Runtime: 2 minutes


> We choose lightweight, cheap, efficient and more simple structured algorithms in order to experiment with and ultimately set our Baseline model.

> We conclude that the best classifier, for our Baseline Model would **Linear SVC**, who except of each descent performance based on our metrics, he's incredibly fast and time efficient.

### Best Classifier

#### Cross Validation

In [29]:
hamming_losses, micro_f1_scores, macro_f1_scores = [], [], []
kf = KFold(n_splits=5)

# Perform manual k-fold cross-validation
for fold_idx, (train_index, test_index) in enumerate(kf.split(X_train), 1):
    X_train_fold, X_test_fold = X_train[train_index], X_train[test_index]
    Y_train_fold, Y_test_fold = Y_train[train_index], Y_train[test_index]

    # Train and Predict
    svc_model.fit(X_train_fold, Y_train_fold)
    Y_pred_fold = svc_model.predict(X_test_fold)

    # Each Fold's Metrics
    hamming_loss_fold = round(hamming_loss(Y_test_fold, Y_pred_fold), 2)
    micro_f1_fold = round(f1_score(Y_test_fold, Y_pred_fold, average='micro'), 2)
    macro_f1_fold = round(f1_score(Y_test_fold, Y_pred_fold, average='macro'), 2)

    # Store metrics
    hamming_losses.append(hamming_loss_fold)
    micro_f1_scores.append(micro_f1_fold)
    macro_f1_scores.append(macro_f1_fold)

    print("-" * 30)
    print(f"KFold {fold_idx}:")
    print(f"\tHamming Loss: {hamming_loss_fold}")
    print(f"\tMicro-F1: {micro_f1_fold}")
    print(f"\tMacro-F1: {macro_f1_fold}")

print("-" * 30 + '\n')
print("-" * 90)
print(f"CV Mean Hamming Loss: {round(np.mean(hamming_losses), 2)}\tCV Mean Micro-F1: {round(np.mean(micro_f1_scores), 2)}\t\tCV Mean Macro-F1: {round(np.mean(macro_f1_scores), 2)}")
print("-" * 90)

------------------------------
KFold 1:
	Hamming Loss: 0.03
	Micro-F1: 0.83
	Macro-F1: 0.8
------------------------------
KFold 2:
	Hamming Loss: 0.03
	Micro-F1: 0.83
	Macro-F1: 0.81
------------------------------
KFold 3:
	Hamming Loss: 0.03
	Micro-F1: 0.82
	Macro-F1: 0.81
------------------------------
KFold 4:
	Hamming Loss: 0.03
	Micro-F1: 0.83
	Macro-F1: 0.81
------------------------------
KFold 5:
	Hamming Loss: 0.03
	Micro-F1: 0.83
	Macro-F1: 0.8
------------------------------

------------------------------------------------------------------------------------------
CV Mean Hamming Loss: 0.03	CV Mean Micro-F1: 0.83		CV Mean Macro-F1: 0.81
------------------------------------------------------------------------------------------


#### Classification Report for each Tag

In [30]:
mlb = MultiLabelBinarizer()
tag_matrix = mlb.fit_transform(df['Tag'].tolist())

> Use MultiLabelBinarizer to retrieve Tag labels.

In [31]:
print('*'*60)
for i in range(Y_train.shape[1]):
    print(mlb.classes_[i])
    print(classification_report(Y_test[:,i], Y_pred[:,i]),'\n'+'*'*60)

************************************************************
android
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5782
           1       0.99      0.92      0.95       900

    accuracy                           0.99      6682
   macro avg       0.99      0.96      0.97      6682
weighted avg       0.99      0.99      0.99      6682
 
************************************************************
angularjs
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6509
           1       0.99      0.95      0.97       173

    accuracy                           1.00      6682
   macro avg       1.00      0.98      0.99      6682
weighted avg       1.00      1.00      1.00      6682
 
************************************************************
c
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6490
           1       0.83      0.6

In [32]:
print("Total time to run the notebook:", int((time.time()-start_time)/60), "minutes")

Total time to run the notebook: 3 minutes
